Übung: Broker und Filter mit K-native
-----------------------------

![](https://user-images.githubusercontent.com/16281246/116248768-1fe56080-a73a-11eb-9a85-8bdccb82d16c.png)

Quelle: K-native

- - - 

**Broker**
Ein Broker in Knative ist eine zentrale Komponente, die als Verteiler von Ereignissen (Events) fungiert. Er empfängt Ereignisse von verschiedenen Quellen und sorgt dafür, dass diese an die richtigen Abonnenten weitergeleitet werden. Ein Broker besteht aus zwei Hauptteilen:

* Ingress: Hier kommen die Ereignisse an.
* Channel: Ein internes Kommunikationssystem, das die Ereignisse an die Abonnenten weiterleitet.

**Filter (Trigger)**
Ein Filter, auch Trigger genannt, definiert eine Regel, die bestimmt, welche Ereignisse an eine bestimmte Knative-Services oder Endpunkte gesendet werden. Triggers filtern die Ereignisse basierend auf bestimmten Kriterien, wie z.B. dem Ereignistyp oder anderen Attributen, und leiten sie dann an den entsprechenden Empfänger weiter.

**Zusammenspiel von Broker und Trigger**
* Ereignisaufnahme: Der Broker empfängt Ereignisse von verschiedenen Quellen.
* Verteilung: Der Broker verteilt die Ereignisse an die entsprechenden Triggers.
* Filterung: Triggers filtern die Ereignisse gemäß den definierten Regeln.
* Zustellung: Gefilterte Ereignisse werden an die entsprechenden Services oder Endpunkte zugestellt.

Durch diese Architektur ermöglicht Knative eine lose Kopplung von Ereignisquellen und -empfängern, was die Skalierbarkeit und Flexibilität von serverlosen Anwendungen erhöht.

- - -

Zuerst erstellen wir den Kubernetes Namespace



In [ ]:
import os
os.environ['NS_BRKR']='ms-brkr'
! kubectl create namespace ${NS_BRKR}
! # kubectl label  namespace ${NS_BRKR} istio-injection=enabled

### Dashboard

Jetzt ist ein guter Zeitpunkt um das Kubernetes Dashboard zu starten und dort im Pulldownmenu den Namespace "ms-brkr" auszuwählen.

Wählt nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":8443"

Anschliessend folgen die Standard Microservices

In [ ]:
%%bash
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-deployment.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-deployment.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-service.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-service.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-service.yaml
kubectl apply --namespace ${NS_BRKR} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-service.yaml
kubectl get   --namespace ${NS_BRKR} pods,services

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_BRKR} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

**Sales** starten wir im Serverless Modus und **Shipment** und **Invoicing** mit genau einer Instanz.

In [ ]:
%%bash
kubectl patch configmap/config-domain --namespace knative-serving --type merge --patch '{"data":{"dukmaster-10-default.mshome.net":""}}' 
#kn service create invoicing --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/invoicing:4.0.0 --port 8080 --namespace ${NS_BRKR}
#kn service create shipment  --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/backoffice/shipment:4.0.0  --port 8080 --namespace ${NS_BRKR}
kn service create invoicing --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/cloudevents --port 8080 --namespace ${NS_BRKR}
kn service create shipment  --scale 1 --image registry.gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/cloudevents --namespace ${NS_BRKR}

kn service create sales               --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/management/sales:3.2.0  --port 8080 --namespace ${NS_BRKR}

Um die Services via Browser anzusprechen ist im Windows die Datei `C:\Windows\System32\Drivers\etc\hosts` um folgende Einträge zu ergänzen.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
echo $(cat ~/work/server-ip) shipment.${NS_BRKR}.dukmaster-10-default.mshome.net
echo $(cat ~/work/server-ip) invoicing.${NS_BRKR}.dukmaster-10-default.mshome.net
echo $(cat ~/work/server-ip) sales.${NS_BRKR}.dukmaster-10-default.mshome.net

echo ""
echo "Browser URLs"
# echo http://shipment.${NS_BRKR}.dukmaster-10-default.mshome.net:${PORT}/shipment
# echo http://invoicing.${NS_BRKR}.dukmaster-10-default.mshome.net:${PORT}/invoicing
echo http://shipment.${NS_BRKR}.dukmaster-10-default.mshome.net:${PORT}/
echo http://invoicing.${NS_BRKR}.dukmaster-10-default.mshome.net:${PORT}/

echo http://sales.${NS_BRKR}.dukmaster-10-default.mshome.net:${PORT}/sales

- - -
### Broker

Für die Kommunkation erstellen wir einen Default Broker

In [ ]:
! kn broker create default --namespace ${NS_BRKR}
! kn broker list --namespace ${NS_BRKR}

- - -
### Trigger

Und drei Trigger welche gezielt die Nachrichten mit `type`: `shipment`, `invoicing` und `order` behandeln.

`order` wird an beide Microservices gesendent die anderen nur an die entsprechenden Namensvetter.

In [ ]:
%%bash
cat <<EOF | kubectl apply --namespace ${NS_BRKR} -f -
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: invoicing
spec:
  broker: default
  filter:
    attributes:
      type: invoicing 
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: invoicing
EOF

In [ ]:
%%bash
cat <<EOF | kubectl apply --namespace ${NS_BRKR}  -f -
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: shipment
spec:
  broker: default
  filter:
    attributes:
      type: shipment
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: shipment
EOF

In [ ]:
%%bash

cat <<EOF | kubectl apply --namespace ${NS_BRKR}  -f -
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: invoicing-order
spec:
  broker: default
  filter:
    attributes:
      type: order 
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: invoicing
---
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: shipment-order
spec:
  broker: default
  filter:
    attributes:
      type: order
  subscriber:
    ref:
     apiVersion: serving.knative.dev/v1
     kind: Service
     name: shipment
EOF

### Gesamtübersicht

Die Subscriptions filtern `"Ce-Type:` und leiten die Events weiter an die Microservices **Shipment** und/oder **Invoicing**.

In [ ]:
%%bash
kubectl get broker default -o jsonpath='{.status.address.url}' --namespace ${NS_BRKR}
echo ""
echo ""
kn trigger list --namespace ${NS_BRKR}

Die Aufrufe zum Broker müssen innerhalb eines Containers abgesetzt werden. 

Deshalb starten wir unseren Testcontainer

In [ ]:
%%bash
kubectl run --image=registry.gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/cloudevents curler --namespace ${NS_BRKR}

Zuerst senden wir einen Event an den Container **shipment**

In [ ]:
%%bash
kubectl exec curler --namespace ${NS_BRKR} -- curl -s -X POST http://broker-ingress.knative-eventing.svc.cluster.local/ms-brkr/default \
-H "Ce-Id: say-hello" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: shipment" \
-H "Ce-Source: curl" \
-H "Content-Type: application/json" \
-d '{"key":"shipment"}'

Und kontrollieren, dass nur dieser die Meldung empfangen hat `"Ce-Type: shipment"`

In [ ]:
! kubectl logs deployment/shipment-00001-deployment --namespace ${NS_BRKR} | tail -20
! echo "--------------------"
! kubectl logs deployment/invoicing-00001-deployment --namespace ${NS_BRKR} | tail -20

Dann senden wir einen Event an den Microservice **invoicing**

In [ ]:
%%bash
kubectl exec curler --namespace ${NS_BRKR} -- curl -s -X POST http://broker-ingress.knative-eventing.svc.cluster.local/ms-brkr/default \
-H "Ce-Id: say-hello" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: invoicing" \
-H "Ce-Source: curl" \
-H "Content-Type: application/json" \
-d '{"key":"invoicing"}'

kubectl logs deployment/shipment-00001-deployment --namespace ${NS_BRKR} | tail -20
echo "--------------------"
kubectl logs deployment/invoicing-00001-deployment --namespace ${NS_BRKR} | tail -20

Und zum Schluss an beide Microservices.

In [ ]:
%%bash
kubectl exec curler --namespace ${NS_BRKR} -- curl -s -X POST http://broker-ingress.knative-eventing.svc.cluster.local/ms-brkr/default \
-H "Ce-Id: say-hello" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: order" \
-H "Ce-Source: curl" \
-H "Content-Type: application/json" \
-d '{"key":"invoicing + shipment"}'

kubectl logs deployment/shipment-00001-deployment --namespace ${NS_BRKR} | tail -20
echo "--------------------"
kubectl logs deployment/invoicing-00001-deployment --namespace ${NS_BRKR} | tail -20

- - -

Aufräumen


In [ ]:
! # kubectl delete pod --all --namespace ${NS_BRKR} --grace-period=0 --force
! kn service delete shipment  --namespace ${NS_BRKR}
! kn service delete invoicing --namespace ${NS_BRKR}
! kn service delete sales --namespace ${NS_BRKR}
! kubectl delete namespace ${NS_BRKR}